In [ ]:
# syft absolute
import syft as sy
from syft.service.log.log import SyftLogV3
from syft.types.syft_object import Context
from syft.types.syft_object import SyftObject

In [ ]:
print(f"syft version: {sy.__version__}")

TODOS
- [x] action objects
- [x] maybe an example of how to migrate one object type in a custom way
- [x] check SyftObjectRegistry and compare with current implementation
- [x] run unit tests
- [ ] finalize notebooks for testing, run in CI
- [ ] other tasks defined in tickets

In [ ]:
node = sy.orchestra.launch(
    name="test_upgradability",
    dev_mode=True,
    local_db=True,
    n_consumers=2,
    create_producer=True,
    migrate=False,
)

In [ ]:
client = node.login(email="info@openmined.org", password="changethis")

# Client side migrations

## document store objects

In [ ]:
migration_dict = client.services.migration.get_migration_objects(get_all=True)

In [ ]:
migration_dict

In [ ]:
def custom_migration_function(context, obj: SyftObject, klass) -> SyftObject:
    # Here, we are just doing the same, but this is where you would write your custom logic
    return obj.migrate_to(klass.__version__, context)

In [ ]:
# this wont work in the cases where the context is actually used,
# but since this would need custom logic here anyway you write workarounds for that (manually querying required state)


context = Context()
migrated_objects = []
for klass, objects in migration_dict.items():
    for obj in objects:
        if isinstance(obj, SyftLogV3):
            migrated_obj = custom_migration_function(context, obj, klass)
        else:
            migrated_obj = obj.migrate_to(klass.__version__, context)
        migrated_objects.append(migrated_obj)

In [ ]:
migrated_objects

In [ ]:
res = client.services.migration.update_migrated_objects(migrated_objects)

In [ ]:
assert isinstance(res, sy.SyftSuccess)

## Actions and ActionObjects

In [ ]:
migration_action_dict = client.services.migration.get_migration_actionobjects()

In [ ]:
# this wont work in the cases where the context is actually used, but since this you would need custom logic here anyway
# it doesnt matter
context = Context()
migrated_actionobjects = []
for klass, objects in migration_action_dict.items():
    for obj in objects:
        # custom migration logic here
        migrated_actionobject = obj.migrate_to(klass.__version__, context)
        migrated_actionobjects.append(migrated_actionobject)

In [ ]:
migrated_actionobjects

In [ ]:
res = client.services.migration.update_migrated_objects(migrated_actionobjects)

In [ ]:
assert isinstance(res, sy.SyftSuccess)

## Store metadata

- Permissions
- StoragePermissions

In [ ]:
store_metadata = client.services.migration.get_all_store_metadata()
store_metadata

In [ ]:
for k, v in store_metadata.items():
    if len(v.permissions):
        print(
            k, len(v.permissions), len(v.permissions) == len(migration_dict.get(k, []))
        )

In [ ]:
# Test update method with a temp node
# After update, all metadata should match between the nodes

temp_node = sy.orchestra.launch(
    name="temp_node",
    dev_mode=True,
    local_db=True,
    n_consumers=2,
    create_producer=True,
    migrate=False,
    reset=True,
)

temp_client = temp_node.login(email="info@openmined.org", password="changethis")

In [ ]:
temp_client.services.migration.update_store_metadata(store_metadata)

In [ ]:
for cname, real_partition in node.python_node.document_store.partitions.items():
    temp_partition = temp_node.python_node.document_store.partitions[cname]

    temp_perms = dict(temp_partition.permissions.items())
    real_perms = dict(real_partition.permissions.items())

    # Only look at migrated items
    temp_perms = {k: v for k, v in temp_perms.items() if k in real_perms}
    assert temp_perms == real_perms

    temp_storage = dict(temp_partition.storage_permissions.items())
    real_storage = dict(real_partition.storage_permissions.items())
    temp_storage = {k: v for k, v in temp_storage.items() if k in real_storage}

    assert temp_storage == real_storage

# Action store
real_partition = node.python_node.action_store
temp_partition = temp_node.python_node.action_store
temp_perms = dict(temp_partition.permissions.items())
real_perms = dict(real_partition.permissions.items())

# Only look at migrated items
temp_perms = {k: v for k, v in temp_perms.items() if k in real_perms}
assert temp_perms == real_perms

temp_storage = dict(temp_partition.storage_permissions.items())
real_storage = dict(real_partition.storage_permissions.items())
temp_storage = {k: v for k, v in temp_storage.items() if k in real_storage}

assert temp_storage == real_storage